## 5. Deploying Machine Learning Models

## Question 1: Version of Pipenv

In [ ]:
# !pip install pipenv
!pipenv --version

pipenv, version 2021.5.29


## Question 2: Checksum for Scikit-Learn 1.0

~~~bash
python3 -m pipenv install scikit-learn==1.0
grep -A 2 "scikit-learn" Pipfile.lock

"scikit-learn": {
            "hashes": [
    "sha256:121f78d6564000dc5e968394f45aac87981fcaaf2be40cfcd8f07b2baa1e1829",
~~~

## Question 3: Probability of churning (Script)

In [ ]:
# !PREFIX=https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/05-deployment/homework
# !wget $PREFIX/model1.bin
# !wget $PREFIX/dv.bin

In [ ]:
import pickle
import sklearn

with open('model1.bin', 'rb') as f_in_model:
    model = pickle.load(f_in_model)

In [ ]:
with open('dv.bin', 'rb') as f_in_dv:
    dv = pickle.load(f_in_dv)

In [9]:
!md5sum model1.bin dv.bin

5868e129bfbb309ba60bf750263afab1  model1.bin
c49b69f8a5a3c560882ff5daa3c0ff4d  dv.bin


In [10]:
dv, model

(DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
                sparse=False),
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False))

In [17]:
customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 19.7}

In [18]:
X = dv.transform(customer)

In [21]:
round(model.predict_proba(X)[0,1],3)

0.115

## Question 4: Probability of churning (Flask)


In [ ]:
# !pip install flask

In [ ]:
import requests

url = "http://127.0.0.1:9696/predict"
customer = {"contract": "two_year", "tenure": 1, "monthlycharges": 10}
result = requests.post(url, json=customer).json()

print(result)

~~~bash
{'churn_probability': 0.999}
~~~

## Question 5: Digest for the base image (Docker)


~~~bash
sudo docker build -t churn-prediction .
Sending build context to Docker daemon  20.48kB
Step 1/8 : FROM agrigorev/zoomcamp-model:3.8.12-slim
3.8.12-slim: Pulling from agrigorev/zoomcamp-model
bd897bb914af: Already exists
aee78d822213: Already exists
6d9f6b5c1e71: Already exists
cf9f290bd6be: Already exists
5e4b501cbda5: Pulling fs layer
5e4b501cbda5: Pull complete
bd464adb9682: Pull complete
c803b748156d: Pull complete
Digest: sha256:1ee036b365452f8a1da0dbc3bf5e7dd0557cfd33f0e56b28054d1dbb9c852023
Status: Downloaded newer image for agrigorev/zoomcamp-model:3.8.12-slim
 ---> f0f43f7bc6e0
 ...
~~~


~~~bash
 sudo docker build -t churn-prediction .
Sending build context to Docker daemon  20.48kB
Step 1/10 : FROM agrigorev/zoomcamp-model:3.8.12-slim
 ---> f0f43f7bc6e0
Step 2/10 : RUN pip install pipenv
 ---> Using cache
 ---> 22e989fec974
Step 3/10 : RUN pip install gunicorn
 ---> Using cache
 ---> 1810590b1122
Step 4/10 : RUN pip install flask
 ---> Using cache
 ---> c3c1c0acc065
Step 5/10 : WORKDIR /app
 ---> Using cache
 ---> 49a126692b6c
Step 6/10 : COPY ["Pipfile", "Pipfile.lock", "./"]
 ---> Using cache
 ---> 9967eab1740b
Step 7/10 : RUN pipenv install --deploy --system
 ---> Using cache
 ---> a2be231a0f96
Step 8/10 : COPY ["predict_app.py", "*.bin", "./"]
 ---> ffd1407c79e0
Step 9/10 : EXPOSE 9696
 ---> Running in a7dddb804315
Removing intermediate container a7dddb804315
 ---> 3294e057ebbb
Step 10/10 : ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9696", "predict_app:app"]
 ---> Running in a31c99a70378
Removing intermediate container a31c99a70378
 ---> 5fc4b0544508
Successfully built 5fc4b0544508
Successfully tagged churn-prediction:latest
~~~

## Question 6: Probability of churning (Docker)

~~~bash
sudo docker run -it -p 9696:9696 churn-prediction:latest
~~~

~~~bash
python3 test_predict.py

{'churn_probability': 0.329}
~~~